In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten , Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers, activations
import tensorflow.keras.backend as K
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import tensorflow_probability as tfp

import numpy as np
import matplotlib.pyplot as plt
import corner
import os 
import sys
import time
import importlib
import logging
from tqdm import tqdm

importlib.reload(logging)
logging.basicConfig(level = logging.INFO)

# limit GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [2]:
training_data = np.load("./nsi_data/sample_nsi_regression_1e7_v1.npz")
data_all = np.column_stack([training_data['ve_dune'][:,:36], training_data['vu_dune'][:,:36], training_data['vebar_dune'][:,:36], training_data['vubar_dune'][:,:36]])

target = np.column_stack([training_data["theta13"], training_data["theta23"],
                          np.sin(training_data["delta"]/180*np.pi), np.cos(training_data["delta"]/180*np.pi),
                         training_data["mumu"], training_data["emu"],
                         training_data["etau"]])

x_train = data_all
y_train = target
x_train_poisson = np.random.poisson(x_train)/1000

In [3]:
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [4]:
latent_dim = 7

"""
Encoder 1 (parameter + spectrum)
"""
encoder_parameter_inputs = layers.Input(shape=(len(y_train[0]),),name = 'encoder_parameter_inputs')
x_parameter = layers.Dense(64, activation="relu", name = 'dense_parameter_1')(encoder_parameter_inputs)
x_parameter = layers.Dense(32, activation="relu", name = 'dense_parameter_2')(x_parameter)
x_parameter = layers.Dense(16, activation="relu", name = 'dense_parameter_3')(x_parameter)

encoder_spectrum_inputs = layers.Input(shape=(144),name = 'encoder_spectrum_inputs')
x_spectrum = layers.Dense(64, activation="relu", name = 'dense_spectrum_1')(encoder_spectrum_inputs)
x_spectrum = layers.Dense(32, activation="relu", name = 'dense_spectrum_2')(x_spectrum)
x_spectrum = layers.Dense(16, activation="relu", name = 'dense_spectrum_3')(x_spectrum)

mergedOut_Encoder_1 = Concatenate()([x_parameter,x_spectrum])

z_mean = layers.Dense(latent_dim, name="z_mean")(mergedOut_Encoder_1)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(mergedOut_Encoder_1)

encoder_1 = keras.Model([encoder_parameter_inputs, encoder_spectrum_inputs], [z_mean, z_log_var], name="encoder_1")
encoder_1.summary()

Model: "encoder_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_parameter_inputs (Input [(None, 7)]          0                                            
__________________________________________________________________________________________________
encoder_spectrum_inputs (InputL [(None, 144)]        0                                            
__________________________________________________________________________________________________
dense_parameter_1 (Dense)       (None, 64)           512         encoder_parameter_inputs[0][0]   
__________________________________________________________________________________________________
dense_spectrum_1 (Dense)        (None, 64)           9280        encoder_spectrum_inputs[0][0]    
__________________________________________________________________________________________

In [5]:
"""
Encoder 2 (spectrum)
"""
encoder_spectrum_inputs = layers.Input(shape=(len(x_train[0]),),name = 'encoder_spectrum_inputs')
x_spectrum = layers.Dense(64, activation="relu", name = 'dense_spectrum_1')(encoder_spectrum_inputs)
x_spectrum = layers.Dense(32, activation="relu", name = 'dense_spectrum_2')(x_spectrum)
x_spectrum = layers.Dense(16, activation="relu", name = 'dense_spectrum_3')(x_spectrum)

guassian_number = 40
z_mean = layers.Dense(guassian_number*latent_dim, name="z_mean")(x_spectrum)
z_log_var = layers.Dense(guassian_number*latent_dim, name="z_log_var")(x_spectrum)
z_weight = layers.Dense(guassian_number, name="z_weight")(x_spectrum)

encoder_2 = keras.Model(encoder_spectrum_inputs, [z_mean, z_log_var, z_weight], name="encoder_2")
encoder_2.summary()

Model: "encoder_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_spectrum_inputs (InputL [(None, 144)]        0                                            
__________________________________________________________________________________________________
dense_spectrum_1 (Dense)        (None, 64)           9280        encoder_spectrum_inputs[0][0]    
__________________________________________________________________________________________________
dense_spectrum_2 (Dense)        (None, 32)           2080        dense_spectrum_1[0][0]           
__________________________________________________________________________________________________
dense_spectrum_3 (Dense)        (None, 16)           528         dense_spectrum_2[0][0]           
__________________________________________________________________________________________

In [6]:
"""
Decoder Model (latent + spectrum)
"""
latent_dim_2 = len(y_train[0])

decoder_latent_inputs = keras.Input(shape=(latent_dim,),name = 'decoder_latent_inputs')
x_latent = layers.Dense(64, activation="relu", name = 'dense_1')(decoder_latent_inputs)
x_latent = layers.Dense(32, activation="relu", name = 'dense_2')(x_latent)
x_latent = layers.Dense(16, activation="relu", name = 'dense_3')(x_latent)

# spectrum
decoder_spectrum_inputs = layers.Input(shape=(144,),name = 'decoder_spectrum_inputs')
x_spectrum = layers.Dense(64, activation="relu", name = 'dense_spectrum_1')(decoder_spectrum_inputs)
x_spectrum = layers.Dense(32, activation="relu", name = 'dense_spectrum_2')(x_spectrum)
x_spectrum = layers.Dense(16, activation="relu", name = 'dense_spectrum_3')(x_spectrum)

mergedOut_Decoder = Concatenate()([x_latent,x_spectrum])

z2_mean = layers.Dense(latent_dim_2, name="z_mean")(mergedOut_Decoder)
z2_log_var = layers.Dense(latent_dim_2, name="z_log_var")(mergedOut_Decoder)

decoder = keras.Model([decoder_latent_inputs, decoder_spectrum_inputs], [z2_mean, z2_log_var], name="decoder")
decoder.summary()

Model: "decoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_latent_inputs (InputLay [(None, 7)]          0                                            
__________________________________________________________________________________________________
decoder_spectrum_inputs (InputL [(None, 144)]        0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           512         decoder_latent_inputs[0][0]      
__________________________________________________________________________________________________
dense_spectrum_1 (Dense)        (None, 64)           9280        decoder_spectrum_inputs[0][0]    
____________________________________________________________________________________________

In [7]:
class CVAE(keras.Model):
    def __init__(self, encoder1, encoder2, decoder, **kwargs):
        super(CVAE, self).__init__(**kwargs)
        self.encoder1 = encoder1
        self.encoder2 = encoder2
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
                ]

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            SMALL_CONSTANT = 1e-12
            
            z1_mean, z1_log_var = self.encoder1(x)
            
            temp_var_q = SMALL_CONSTANT + tf.exp(z1_log_var)
            mvn_q = tfp.distributions.MultivariateNormalDiag(
                          loc=z1_mean,
                          scale_diag=tf.sqrt(temp_var_q))
            
            z1 = mvn_q.sample()
            
            z2_mean, z2_log_var, z2_weight = self.encoder2(x[1])

            z2_mean = tf.reshape(z2_mean, (-1, guassian_number, latent_dim))
            z2_log_var = tf.reshape(z2_log_var, (-1, guassian_number, latent_dim))
            z2_weight = tf.reshape(z2_weight, (-1, guassian_number))

            temp_var_r1 = SMALL_CONSTANT + tf.exp(z2_log_var)
            bimix_gauss = tfp.distributions.MixtureSameFamily(
                          mixture_distribution=tfp.distributions.Categorical(logits=z2_weight),
                          components_distribution=tfp.distributions.MultivariateNormalDiag(
                          loc=z2_mean,
                          scale_diag=tf.sqrt(temp_var_r1)))
            
            z2 = bimix_gauss.sample()
            
            reconstruction_mean, reconstruction_var = self.decoder([z1, x[1]])     
            
            temp_var_r2 = SMALL_CONSTANT + tf.exp(reconstruction_var)
            reconstruction_parameter = tfp.distributions.MultivariateNormalDiag(
                                     loc=reconstruction_mean,
                                     scale_diag= tf.sqrt(temp_var_r2))
            
            r2 = reconstruction_parameter.sample()

            log_q_q = mvn_q.log_prob(z1)
            log_r1_q = bimix_gauss.log_prob(z1)
            kl_loss = tf.reduce_mean(log_q_q - log_r1_q)
            
            reconstruction_parameter_loss = reconstruction_parameter.log_prob(y)
            reconstruction_loss = -1.0*tf.reduce_mean(reconstruction_parameter_loss)
            
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [8]:
"""
Model Building
"""
cvae = CVAE(encoder_1,encoder_2,decoder)
cvae.compile(optimizer=keras.optimizers.Adam())

In [ ]:
for i in range(2, 11):
    training_data = np.load("./nsi_data/sample_nsi_regression_1e7_v{}.npz".format(i))
    data_all = np.column_stack([training_data['ve_dune'][:,:36], training_data['vu_dune'][:,:36], training_data['vebar_dune'][:,:36], training_data['vubar_dune'][:,:36]])

    target = np.column_stack([training_data["theta13"], training_data["theta23"],
                              np.sin(training_data["delta"]/180*np.pi), np.cos(training_data["delta"]/180*np.pi),
                             training_data["mumu"], training_data["emu"],
                             training_data["etau"]])

    x_train = data_all
    y_train = target
    x_train_poisson = np.random.poisson(x_train)/1000
    cvae.fit(x = [y_train, x_train_poisson],
             y = y_train,
             batch_size=1000,
             epochs=300,
             verbose=1,
             shuffle=True)

Epoch 1/300
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it with `scale_diag` directly instead.
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
10000/10000 [==============================] - 67s 7ms/step - loss: 15.6887 - reconstruction_loss: 6.7993 - kl_loss: 0.2865
Epoch 2/300
10000/10000 [==============================] - 65s 7ms/step - loss: 3.8832 - reconstruction_loss: 3.6113 - kl_loss: 0.0613
Epoch 3/300
10000/10000 [==============================] - 66s 7ms/step - loss: 3.2472 - reconstruction_loss: 2.7928 - kl_loss: 0.3307
Epoch 4/300
10000/10000 [==============================] - 66s 7ms/step - loss: 2.6749 - reconstruction_loss: 1.2148 - kl_loss: 1.3168
Epoch 5/300
10000/10000 [==============================] - 67s 7ms/step - loss: 2.1750 - reconstruction_loss: 0.5354 - kl_loss: 1.5617
Epoch 6/300
10000/10000 [==============================] - 66s 7ms/step - loss: 1.9076 - reconstruction_loss: 0.2

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



10000/10000 [==============================] - 66s 7ms/step - loss: 0.3746 - reconstruction_loss: -1.3187 - kl_loss: 1.6807
Epoch 45/300
10000/10000 [==============================] - 65s 7ms/step - loss: 0.3432 - reconstruction_loss: -1.3625 - kl_loss: 1.6990
Epoch 46/300
 8345/10000 [========================>.....] - ETA: 10s - loss: 0.3386 - reconstruction_loss: -1.3627 - kl_loss: 1.6952

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



10000/10000 [==============================] - 66s 7ms/step - loss: 0.2998 - reconstruction_loss: -1.3897 - kl_loss: 1.6921
Epoch 53/300
10000/10000 [==============================] - 65s 7ms/step - loss: 0.4021 - reconstruction_loss: -1.3771 - kl_loss: 1.7037
Epoch 54/300
10000/10000 [==============================] - 66s 7ms/step - loss: 0.2401 - reconstruction_loss: -1.4583 - kl_loss: 1.6964
Epoch 55/300
 4199/10000 [===========>..................] - ETA: 38s - loss: 0.2174 - reconstruction_loss: -1.4853 - kl_loss: 1.7044

...

10000/10000 [==============================] - 65s 7ms/step - loss: 0.0245 - reconstruction_loss: -1.6780 - kl_loss: 1.7035
Epoch 89/300
10000/10000 [==============================] - 64s 6ms/step - loss: 0.0133 - reconstruction_loss: -1.6944 - kl_loss: 1.7068
Epoch 90/300
10000/10000 [==============================] - 64s 6ms/step - loss: 0.0112 - reconstruction_loss: -1.6913 - kl_loss: 1.7063
Epoch 91/300
 3452/10000 [=========>....................] - ETA: 41s - loss: 0.0389 - reconstruction_loss: -1.6973 - kl_loss: 1.7034

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



10000/10000 [==============================] - 66s 7ms/step - loss: 0.0056 - reconstruction_loss: -1.6976 - kl_loss: 1.6998
Epoch 93/300
10000/10000 [==============================] - 66s 7ms/step - loss: 0.0025 - reconstruction_loss: -1.7006 - kl_loss: 1.7136
Epoch 94/300
10000/10000 [==============================] - 66s 7ms/step - loss: 0.0133 - reconstruction_loss: -1.6974 - kl_loss: 1.7095
Epoch 95/300
10000/10000 [==============================] - 65s 7ms/step - loss: -4.1920e-04 - reconstruction_loss: -1.6946 - kl_loss: 1.6909
Epoch 96/300
10000/10000 [==============================] - 65s 7ms/step - loss: 0.0043 - reconstruction_loss: -1.6909 - kl_loss: 1.6919
Epoch 97/300
10000/10000 [==============================] - 65s 6ms/step - loss: -0.0077 - reconstruction_loss: -1.6987 - kl_loss: 1.7004
Epoch 98/300
10000/10000 [==============================] - 65s 7ms/step - loss: -0.0185 - reconstruction_loss: -1.7299 - kl_loss: 1.7224
Epoch 99/300
10000/10000 [=====================

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



10000/10000 [==============================] - 66s 7ms/step - loss: -0.1126 - reconstruction_loss: -1.8445 - kl_loss: 1.7303
Epoch 141/300
10000/10000 [==============================] - 66s 7ms/step - loss: -0.1126 - reconstruction_loss: -1.8315 - kl_loss: 1.7153
Epoch 142/300
10000/10000 [==============================] - 66s 7ms/step - loss: -0.1231 - reconstruction_loss: -1.8496 - kl_loss: 1.7270
Epoch 144/300
10000/10000 [==============================] - 64s 6ms/step - loss: -0.1264 - reconstruction_loss: -1.8598 - kl_loss: 1.7285
Epoch 147/300
10000/10000 [==============================] - 65s 7ms/step - loss: -0.1361 - reconstruction_loss: -1.8579 - kl_loss: 1.7255
Epoch 148/300
10000/10000 [==============================] - 65s 7ms/step - loss: -0.1447 - reconstruction_loss: -1.8649 - kl_loss: 1.7304
Epoch 149/300
10000/10000 [==============================] - 65s 7ms/step - loss: -0.1443 - reconstruction_loss: -1.8834 - kl_loss: 1.7424
Epoch 150/300
10000/10000 [==============

In [ ]:
"""
Save file
"""
path = './CVAE/DNN_angleTran_allparam_{},{}'.format(guassian_number, latent_dim) + '_{}'
index = 1
while os.path.isdir(path.format(index)):
    index += 1
path = path.format(index)
cvae.encoder1.save(path + "/encoder_1.h5")
cvae.encoder2.save(path + "/encoder_2.h5")
cvae.decoder.save(path + "/decoder.h5")